In [3]:
!pip uninstall torch torchaudio -y

Found existing installation: torch 2.3.1
Uninstalling torch-2.3.1:
  Successfully uninstalled torch-2.3.1


In [4]:
!pip install autogluon.multimodal

  Using cached torch-2.3.1-cp310-cp310-manylinux1_x86_64.whl.metadata (26 kB)
Using cached torch-2.3.1-cp310-cp310-manylinux1_x86_64.whl (779.1 MB)


In [1]:
!wget --quiet https://automl-mm-bench.s3.amazonaws.com/multilingual-datasets/amazon_review_sentiment_cross_lingual.zip -O amazon_review_sentiment_cross_lingual.zip
!unzip -q -o amazon_review_sentiment_cross_lingual.zip -d .

In [2]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

train_de_df = pd.read_csv('amazon_review_sentiment_cross_lingual/de_train.tsv',
                          sep='\t', header=None, names=['label', 'text']) \
                .sample(1000, random_state=123)
train_de_df.reset_index(inplace=True, drop=True)

test_de_df = pd.read_csv('amazon_review_sentiment_cross_lingual/de_test.tsv',
                          sep='\t', header=None, names=['label', 'text']) \
               .sample(200, random_state=123)
test_de_df.reset_index(inplace=True, drop=True)
print(train_de_df)

     label                                               text
0        0  Dieser Film, nur so triefend von Kitsch, ist h...
1        0  Wie so oft: Das Buch begeistert, der Film entt...
2        1  Schon immer versuchten Männer ihre Gefühle geg...
3        1  Wenn man sich durch 10 Minuten Disney-Trailer ...
4        1  Eine echt geile nummer zum Abtanzen und feiern...
..     ...                                                ...
995      0  Ich dachte dies wäre ein richtig spannendes Bu...
996      0  Wer sich den Schrott wirklich noch ansehen möc...
997      0  Sicher, der Film greift ein aktuelles und hoch...
998      1  Dieser Bildband lässt das Herz von Sarah Kay-F...
999      1  ...so das war nun mein drittes Buch von Jenny-...

[1000 rows x 2 columns]


In [3]:
train_en_df = pd.read_csv('amazon_review_sentiment_cross_lingual/en_train.tsv',
                          sep='\t',
                          header=None,
                          names=['label', 'text']) \
                .sample(1000, random_state=123)
train_en_df.reset_index(inplace=True, drop=True)

test_en_df = pd.read_csv('amazon_review_sentiment_cross_lingual/en_test.tsv',
                          sep='\t',
                          header=None,
                          names=['label', 'text']) \
               .sample(200, random_state=123)
test_en_df.reset_index(inplace=True, drop=True)
print(train_en_df)

     label                                               text
0        0  This is a film that literally sees little wron...
1        0  This music is pretty intelligent, but not very...
2        0  One of the best pieces of rock ever recorded, ...
3        0  Reading the posted reviews here, is like revis...
4        1  I've just finished page 341, the last page. It...
..     ...                                                ...
995      1  This album deserves to be (at least) as popula...
996      1  This book, one of the few that takes a more ac...
997      1  I loved it because it really did show Sagan th...
998      1  Stuart Gordons "DAGON" is a unique horror gem ...
999      0  I've heard Al Lee speak before and thought tha...

[1000 rows x 2 columns]


In [4]:
from autogluon.multimodal import MultiModalPredictor

predictor = MultiModalPredictor(label='label')
predictor.fit(train_de_df,
              hyperparameters={
                  'model.hf_text.checkpoint_name': 'bert-base-german-cased',
                  'optimization.max_epochs': 2
              }, time_limit=900)

No path specified. Models will be saved in: "AutogluonModels/ag-20240915_020725"
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Pytorch Version:    2.3.1+cu121
CUDA Version:       CUDA is not available
Memory Avail:       10.30 GB / 12.67 GB (81.3%)
Disk Space Avail:   63.70 GB / 107.72 GB (59.1%)
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during Predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression', 'quantile'])

AutoMM starts to create your model. ✨✨✨

To track the learning progress, you can open a terminal and launch Tensorboard:
    ```shell
    # Assume 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

INFO: Time limit reached. Elapsed time is 0:15:25. Signaling Trainer to stop.


Validation: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 0, global step 1: 'val_roc_auc' reached 0.60337 (best 0.60337), saving model to '/content/AutogluonModels/ag-20240915_020725/epoch=0-step=1.ckpt' as top 3
AutoMM has created your model. 🎉🎉🎉

To load the model, use the code below:
    ```python
    from autogluon.multimodal import MultiModalPredictor
    predictor = MultiModalPredictor.load("/content/AutogluonModels/ag-20240915_020725")
    ```

If you are not satisfied with the model, try to increase the training time, 
adjust the hyperparameters (https://auto.gluon.ai/stable/tutorials/multimodal/advanced_topics/customization.html),
or post issues on GitHub (https://github.com/autogluon/autogluon/issues).




In [5]:
score = predictor.evaluate(test_de_df)
print('Score on the German Testset:')
print(score)

Predicting: |          | 0/? [00:00<?, ?it/s]

Score on the German Testset:
{'roc_auc': 0.6138822115384615}


In [6]:
score = predictor.evaluate(test_en_df)
print('Score on the English Testset:')
print(score)

Predicting: |          | 0/? [00:00<?, ?it/s]

Score on the English Testset:
{'roc_auc': 0.5699929655311025}


In [4]:
from autogluon.multimodal import MultiModalPredictor

predictor = MultiModalPredictor(label='label')
predictor.fit(train_en_df,
              presets='multilingual',
              hyperparameters={
                  'optimization.max_epochs': 2
              }, time_limit=500)

No path specified. Models will be saved in: "AutogluonModels/ag-20240915_030629"
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Pytorch Version:    2.3.1+cu121
CUDA Version:       CUDA is not available
Memory Avail:       10.60 GB / 12.67 GB (83.6%)
Disk Space Avail:   63.28 GB / 107.72 GB (58.7%)
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during Predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression', 'quantile'])

AutoMM starts to create your model. ✨✨✨

To track the learning progress, you can open a terminal and launch Tensorboard:
    ```shell
    # Assume 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

INFO: Time limit reached. Elapsed time is 0:08:53. Signaling Trainer to stop.


Validation: |          | 0/? [00:00<?, ?it/s]

AutoMM has created your model. 🎉🎉🎉

To load the model, use the code below:
    ```python
    from autogluon.multimodal import MultiModalPredictor
    predictor = MultiModalPredictor.load("/content/AutogluonModels/ag-20240915_030629")
    ```

If you are not satisfied with the model, try to increase the training time, 
adjust the hyperparameters (https://auto.gluon.ai/stable/tutorials/multimodal/advanced_topics/customization.html),
or post issues on GitHub (https://github.com/autogluon/autogluon/issues).




In [5]:
score_in_en = predictor.evaluate(test_en_df)
print('Score in the English Testset:')
print(score_in_en)

Predicting: |          | 0/? [00:00<?, ?it/s]

Score in the English Testset:
{'roc_auc': 0.5894884936187317}


In [6]:
score_in_de = predictor.evaluate(test_de_df)
print('Score in the German Testset:')
print(score_in_de)

Predicting: |          | 0/? [00:00<?, ?it/s]

Score in the German Testset:
{'roc_auc': 0.523036858974359}


In [7]:
test_jp_df = pd.read_csv('amazon_review_sentiment_cross_lingual/jp_test.tsv',
                          sep='\t', header=None, names=['label', 'text']) \
               .sample(200, random_state=123)
test_jp_df.reset_index(inplace=True, drop=True)
print(test_jp_df)

     label                                               text
0        1  原作はビクトル・ユーゴの長編小説だが、私が子供の頃読んだのは短縮版の「ああ無情」。それでもこ...
1        1  ほかの作品のレビューにみんな書いているのに、何故この作品について書いている人が一人しかいない...
2        0  一番の問題点は青島が出ていない事でしょう。  ＴＶ番組では『芸人が出ていればバラエティだから...
3        0  昔、 りんたろう監督によるアニメ「カムイの剣」があった。  「カムイの剣」…を観た人なら本作...
4        1  以前のアルバムを聴いていないのでなんとも言えないが、クラシックなメタルを聞いてきた耳には、と...
..     ...                                                ...
195      0  原作が面白く、このDVDも期待して観ただけに非常にがっかりしました。  脚本としては単に格闘...
196      0                              フェードインやフェードアウトが多すぎます。
197      0  流通形態云々については特に革命と言う気はしない。  これからもＣＤは普通に発売されるだろうし...
198      1  もうＴＶとか、最近の映画とか、観なくていいよ。  脳に楽なエンターテイメントだから。  脳を...
199      0  みんなさんは、手塚治虫先生の「1985への出発」という漫画を読んだことがありますでしょうか？...

[200 rows x 2 columns]


In [8]:
print('Negative labe ratio of the Japanese Testset=', test_jp_df['label'].value_counts()[0] / len(test_jp_df))
score_in_jp = predictor.evaluate(test_jp_df)
print('Score in the Japanese Testset:')
print(score_in_jp)

Negative labe ratio of the Japanese Testset= 0.575


Predicting: |          | 0/? [00:00<?, ?it/s]

Score in the Japanese Testset:
{'roc_auc': 0.5270588235294117}
